# Understanding Dataset Structure

[▲ Overview](0.0-Overview.ipynb)

[▶ Loading and Decoding Dataset](2.0-Loading-dataset.ipynb)

The dataset obtained from the [Australian Bureau of Statistics](http://stat.data.abs.gov.au) is provided as a [SDMX structure](https://sdmx.org/) in JSON. In this exercise, only a minimal subset of SDMX is required. In particular, multidimensional data is not needed.

In [1]:
import json
from australian_housing import paths

In [2]:
with open(paths.manager.raw_data_file) as f:
    data = json.load(f)

In [3]:
data.keys()

dict_keys(['header', 'dataSets', 'structure'])

`header` shows some metadata and does not need to be parsed. `dataSets` contains the actual data and `structure` explains what dimensions are to be expected. While `dataSets` may contain multiple enties, here only a single one is required.

In [4]:
data['header']

{'id': 'c3e3da5d-c6fc-432e-8d7d-615fa6006ab5',
 'test': False,
 'prepared': '2019-01-23T19:33:45.2325909Z',
 'sender': {'id': 'ABS', 'name': 'Australian Bureau of Statistics'},
 'links': [{'href': 'http://stat.data.abs.gov.au:80/sdmx-json/data/ABS_BA_SA2_ASGS2016/1.9.1...0+1+102+1GSYD+10201.M/all?detail=Full&dimensionAtObservation=AllDimensions&startPeriod=2011-07&endPeriod=2017-07',
   'rel': 'request'}]}

`structure.attributes.observation` could be used for generic parsing and understanding data types. For this exercise only a single datetime column exists which can be identified by its id.

In [5]:
data['structure']['attributes']['observation']

[{'id': 'TIME_FORMAT',
  'name': 'Time Format',
  'values': [{'id': 'P1M', 'name': 'Monthly'}]},
 {'id': 'OBS_STATUS', 'name': 'Observation Status', 'values': []},
 {'id': 'UNIT',
  'name': 'Unit',
  'values': [{'id': 'NUM', 'name': 'Number'}],
  'role': 'UNIT_MEASURE'},
 {'id': 'POWERCODE',
  'name': 'Unit multiplier',
  'default': '0',
  'values': [{'id': '0', 'name': 'Units'}],
  'role': 'UNIT_MULT'},
 {'id': 'REFERENCEPERIOD',
  'name': 'Reference period',
  'values': [],
  'role': 'BASE_PER'}]

`structure.dimensions.observation` contains encoding information for the dimensions of the dataset. This is a key information which has be used for parsing!

In [6]:
observations = data['structure']['dimensions']['observation']
observations

[{'keyPosition': 0,
  'id': 'MEASURE',
  'name': 'Measure',
  'values': [{'id': '1', 'name': 'Total number of dwelling units'}]},
 {'keyPosition': 1,
  'id': 'SECTOR',
  'name': 'Sector of Ownership',
  'values': [{'id': '9', 'name': 'Total Sectors'}]},
 {'keyPosition': 2,
  'id': 'WORK',
  'name': 'Type of work',
  'values': [{'id': '1', 'name': 'New'}]},
 {'keyPosition': 3,
  'id': 'BUILDING_TYPE',
  'name': 'Type of building',
  'values': [{'id': '110', 'name': 'Houses'},
   {'id': '121',
    'name': 'Semi-detached, row or terrace houses, townhouses - One storey'},
   {'id': '122',
    'name': 'Semi-detached, row or terrace houses, townhouses - Two or more storeys'},
   {'id': '120',
    'name': 'Semi-detached, row or terrace houses, townhouses - Total'},
   {'id': '131',
    'name': 'Flats units or apartments - In a one or two storey block'},
   {'id': '132',
    'name': 'Flats units or apartments - In a three storey block'},
   {'id': '133',
    'name': 'Flats units or apartments 

In [7]:
[len(obs['values']) for obs in observations]

[1, 1, 1, 10, 4, 4, 1, 73]

The whole dataset contains 2920 entries of which only 73 will be relevant for this exercise. There is only a single entry in `dataSet`.

In [8]:
len(data['dataSets'][0]['observations'])

2920

In [9]:
len(data['dataSets'])

1

`dataSets.0.observations` is an object where all data dimensions are encoded in the key as colon-separated integers. The values of this object are lists of which only the first entry contains any information (the actual measure).

For decoding the dataset, the keys in `dataSets.0.observations` need to be split by `:` and then decoded using the meta information in `structure.dimensions.observation`. This is implemented in `australian_housing.data.extract_dataframe.AustralianHousingLoader`.

In [10]:
data['dataSets'][0]['observations']

{'0:0:0:0:0:0:0:0': [14.0, 0, None, 0, 0, None],
 '0:0:0:0:0:0:0:1': [17.0, 0, None, 0, 0, None],
 '0:0:0:0:0:0:0:2': [21.0, 0, None, 0, 0, None],
 '0:0:0:0:0:0:0:3': [15.0, 0, None, 0, 0, None],
 '0:0:0:0:0:0:0:4': [15.0, 0, None, 0, 0, None],
 '0:0:0:0:0:0:0:5': [18.0, 0, None, 0, 0, None],
 '0:0:0:0:0:0:0:6': [19.0, 0, None, 0, 0, None],
 '0:0:0:0:0:0:0:7': [14.0, 0, None, 0, 0, None],
 '0:0:0:0:0:0:0:8': [14.0, 0, None, 0, 0, None],
 '0:0:0:0:0:0:0:9': [11.0, 0, None, 0, 0, None],
 '0:0:0:0:0:0:0:10': [22.0, 0, None, 0, 0, None],
 '0:0:0:0:0:0:0:11': [17.0, 0, None, 0, 0, None],
 '0:0:0:0:0:0:0:12': [24.0, 0, None, 0, 0, None],
 '0:0:0:0:0:0:0:13': [25.0, 0, None, 0, 0, None],
 '0:0:0:0:0:0:0:14': [23.0, 0, None, 0, 0, None],
 '0:0:0:0:0:0:0:15': [16.0, 0, None, 0, 0, None],
 '0:0:0:0:0:0:0:16': [28.0, 0, None, 0, 0, None],
 '0:0:0:0:0:0:0:17': [16.0, 0, None, 0, 0, None],
 '0:0:0:0:0:0:0:18': [12.0, 0, None, 0, 0, None],
 '0:0:0:0:0:0:0:19': [19.0, 0, None, 0, 0, None],
 '0:0:0:0:

[▲ Overview](0.0-Overview.ipynb)

[▶ Loading and Decoding Dataset](2.0-Loading-dataset.ipynb)